In [18]:
import pandas as pd
import numpy as np
import math
import time
import os
import requests
import json
import urllib.request
import re
import pickle
import networkx as nx
import string

In [19]:
app_id = '24ac8855'
app_key = 'baa032bb2944756dcf361a18ae7e3ab9'

actual_dictionary = {}

alphabet = set(list(string.ascii_lowercase) + list(string.ascii_uppercase))

def find_defs(diction):
    """
    This function executes when we reach a list containing dictionary data strucutres that may contain several different
    definitions for a given word. It runs through this list, for each list finding keys in the dictionary data
    structure containing the string "definitions" or "short_definitions" and appending the corresponding value 
    (a string) of this key to a master string, which we call stringy. At the end of this function, we use the 
    built-in function set() on the list given to us by the .split() method. The reasons for a set are:
    1) checking membership in sets is efficient in python
    2) sets do not contain duplicate elements, so we are using less space.
    """
    stringy = ''
    for i in diction:
        for k, v in i.items():
            if k == 'definitions' or k == 'short_definitions' or k == 'crossReferenceMarkers':
                stringy += v[0] + ' '
    stringy = stringy.split(' ')
    
    for word in stringy:  
        n = stringy.index(word)
        try:
            # Make sure that first, last element in string is an alphanumeric character
            if word[0] not in alphabet:
                stringy[n] = word[1:]
            if stringy[n][-1] not in alphabet:
                stringy[n] = stringy[n][:-1]
            # sometimes dictionary entries have phrases like "(hello it is me.)
            # this has two non-alphanumeric characters at the end. This check below catches this.
            if stringy[n][-1] not in alphabet:
                stringy[n] = stringy[n][:-1]
        except IndexError:
            pass

    return set(stringy)

def find_definition(word):
    """
    This function takes in a word as input and creates an entry in our dataset consisting of a word-definition pair.
    """
#     wordnet_lemmatizer.lemmatize(word)
    
    # there may be times while looking up words that the dictionary does not contain a word for whatever reason.
    # this chunk of code catches this error.
    try:
#         lemmatizedWord = wordnet_lemmatizer.lemmatize(word).lower()
        url = 'https://od-api.oxforddictionaries.com:443/api/v1/entries/en/' + word.lower()
        r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
        dicty = r.json()
        # # Need to consider words like "won" that have atypical structure...........       
        # this process is ugly, but it works. 
        # The structure of the data returned from the API call is a bit awkward.
        lexCategory = dicty['results'][0]['lexicalEntries'][0]['lexicalCategory']
        if lexCategory == "Noun":
            lexCategory = "n"
        if lexCategory == "Verb":
            lexCategory = "v"
        if lexCategory == "Adjective":
            lexCategory = "a"
        definitions = dicty['results'][0]['lexicalEntries'][0]['entries'][0]['senses']
        if find_defs(definitions) != {''}:
            actual_dictionary[word] = find_defs(definitions)
        else:
            pass
    except json.JSONDecodeError:
        pass
    
def find_contents(diction):
    try:
        for k, v in diction.items():
            print(k, '\n', v, '\n\n')
    except AttributeError:
        for i in diction:
            print(i, '\n\n')

In [20]:
# Defining functions using "Pickle" library that allow us to save python objects as .pkl files that, when loaded again,
# act just like the objects we saved them as without needing to transform them.

# This allows us to save a dictionary instead of needing to reconsruct a dictionary for each info-theoretic calculation.

def save_obj(obj, name):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [24]:
# get word list to be used for dictionary lookup
word_list = [word.strip('\n') for word in open("data/words_alpha.txt", 'r').readlines()]

In [59]:
for word in word_list:
    try:
        find_definition(word)
    except KeyError:
        pass
    # add delay so we don't reach API call/min ceiling.
    time.sleep(.2)
    save_obj(actual_dictionary, 'dictionary')

save_obj(actual_dictionary, 'dictionary')

In [21]:
dicty = load_obj('dictionary')

In [90]:
# find_contents(dicty)

Dictionaries can be represented as directed graphs D = (V, A) (digraphs). The vertices
(i.e. the members of v) are words and the arcs connect defining words to defined words,
that is, there is an arc from word u to word v if u is a word in the definition of v. More-
over, in a complete dictionary, every word is defined by at least one word, so we assume
that there is no word without an incoming arc. A path is a sequence (v 1 , v 2, ..., v k ) of ver-
tices such that (v i , v i+1 ) is an arc for i = 1, 2, ..., k-1. A circuit is a path starting and end-
ing at the same vertex. A graph is called acyclic if it does not contain any circuits.

Let U x V be any subset of words and let u be some given word. We are interested
in computing all words that can be learned through definitions composed only of words
in U. This can be stated recursively as follows: We say that u is definable from U if
all predecessors of u either belong to U or are definable from U. The set of words that
can be defined from U is denoted by Def(U). In particular, if Def (U)∪U = V, then U
is called a grounding set of D. Intuitively, a set U is a grounding set if, provided we
already know the meaning of each word in U, we can learn the meaning of all the
remaining words just by looking up the definitions of the unknown words (in the right
order).

As a first step, we observed that in all dictionaries analyzed so far there exist many
words that are never used in any definition. These words can be removed without chang-
ing the MinSets. This reduction can be done iteratively until no further word can be
removed without leaving any word undefinable from the rest. The resulting subgraph is
what we called the dictionary’s (grounding) Kernel

Next, since we are dealing with directed graphs, we can subdivide the words according
to their strongly connected components (SCCs). Two words u and v are strongly connected
if there exists a path from u to v as well as a path from v to u. 

In [63]:
Dictionary = nx.DiGraph()
Dictionary.add_nodes_from(dicty.keys())

In [64]:
len(Dictionary)

85183

In [27]:
def createEdgesFromDefinitions(graph, dictionary):
    # iterate through the dictionary's words and corresponding definitions
    for word_1, definition_1 in dictionary.items():
        # iterate through the remaining words
        for word_2 in dictionary.keys():
            # check if the remaining words define the word we're currently iterating through
            if word_2 in definition_1 and word_1 != word_2:
                # if so, create a directed edge from the word that defines the original word to the original word
                graph.add_edge(word_2, word_1)

#createEdgesFromDefinitions(Dictionary, dicty)

In [66]:
len(Dictionary.edges)

869180

In [67]:
words_to_remove = []

# this process will reduce the dictionary graph to the kernel
for word in Dictionary:
    # contained is a flag; we set it to False at first
    contained = False
    # iterate through each edge
    for key, value in dicty.items():
        # if the current word in the iteration is in the the definition of another word,
        # flag contained = True and break out of the innermost loop
        if word == edge[0]:
            contained = True
            break
    # otherwise, add the word to the list of words to remove
    if contained == False:
        words_to_remove.append(word)

Kernel = Dictionary.copy()

# remove words that don't define other words from graph
for word in words_to_remove:
    Kernel.remove_node(word)

In [22]:
# save_obj(Dictionary, 'dictionary_graph')
Dictionary_graph = load_obj('dictionary_graph')

# save_obj(Kernel, 'kernel_graph')
Kernel_graph = load_obj('kernel_graph')

print(len(Dictionary_graph), len(Dictionary_graph.edges))
print(len(Kernel_graph), len(Kernel_graph.edges))

85183 869180
27935 353180


In [77]:
# find core

Core = nx.DiGraph()

for (word_1, word_2) in Kernel_graph.edges:
        if word_1 in dicty[word_2] and word_2 in dicty[word_1]:
            Core.add_node(word_1)
            Core.add_node(word_2)
            Core.add_edge(word_1, word_2)
            Core.add_edge(word_2, word_1)

In [23]:
# save_obj(Core, 'core_graph')
Core_graph = load_obj('core_graph')

print(len(Core_graph), len(Core_graph.edges))

3766 5130


[('a', 'class'),
 ('a', 'conversation'),
 ('a', 'first'),
 ('a', 'for'),
 ('a', 'in'),
 ('a', 'indicate'),
 ('a', 'membership'),
 ('a', 'of'),
 ('a', 'or'),
 ('a', 'people'),
 ('a', 'per'),
 ('a', 'person'),
 ('a', 'someone'),
 ('a', 'something'),
 ('a', 'text'),
 ('a', 'the'),
 ('a', 'thing'),
 ('a', 'time'),
 ('a', 'to'),
 ('a', 'with'),
 ('class', 'a'),
 ('class', 'category'),
 ('class', 'type'),
 ('conversation', 'a'),
 ('conversation', 'informal'),
 ('first', 'a'),
 ('first', 'person'),
 ('for', 'a'),
 ('for', 'benefit'),
 ('for', 'exchange'),
 ('for', 'favour'),
 ('for', 'on'),
 ('for', 'period'),
 ('for', 'place'),
 ('for', 'purpose'),
 ('for', 'reason'),
 ('for', 'respect'),
 ('for', 'series'),
 ('in', 'a'),
 ('in', 'activity'),
 ('in', 'before'),
 ('in', 'by'),
 ('in', 'case'),
 ('in', 'condition'),
 ('in', 'else'),
 ('in', 'inclusion'),
 ('in', 'involvement'),
 ('in', 'language'),
 ('in', 'length'),
 ('in', 'medium'),
 ('in', 'occupation'),
 ('in', 'of'),
 ('in', 'or'),
 ('in

In [8]:
Core_defining_power = {}
Kernel_defining_power = {}
Dictionary_defining_power = {}

In [9]:
for word in Kernel_graph:
    Kernel_defining_power[word] = 0
    for value in dicty.values():
        if word in value:
            Kernel_defining_power[word] += 1   

find_contents(Kernel_defining_power)

a 
 47015 


aa 
 1 


aardvark 
 2 


ab 
 1 


aback 
 1 


abactinal 
 1 


abacus 
 4 


abalone 
 2 


abandon 
 14 


abandoned 
 17 


abandonment 
 6 


abasement 
 1 


abash 
 1 


abasia 
 1 


abatement 
 4 


abatis 
 1 


abattoir 
 1 


abaxial 
 1 


abbatial 
 1 


abbey 
 5 


abbess 
 2 


abbot 
 6 


abbreviated 
 5 


abbreviation 
 7 


abdicate 
 1 


abdication 
 1 


abdomen 
 66 


abdominal 
 34 


abducens 
 2 


abducent 
 1 


abduct 
 4 


abduction 
 5 


abductor 
 1 


abecedarian 
 1 


abeyance 
 1 


aberrant 
 2 


aberration 
 10 


abettor 
 1 


abide 
 3 


abience 
 1 


abient 
 1 


ability 
 237 


abiogenesis 
 2 


abiogenist 
 1 


abiological 
 1 


abiotic 
 1 


abyss 
 2 


abyssal 
 1 


abject 
 1 


ablation 
 2 


ablative 
 2 


ablaut 
 2 


able 
 563 


abnegation 
 1 


abnormal 
 130 


abnormality 
 19 


abnormally 
 83 


aboard 
 1 


abode 
 8 


abolish 
 4 


abolition 
 9 


abomasum 
 4 


abominable 
 3 


aboral

anagram 
 2 


anal 
 24 


analcime 
 2 


analcite 
 1 


analemma 
 2 


analepsis 
 1 


analeptic 
 2 


analgesic 
 17 


analyse 
 18 


analysis 
 80 


analyst 
 3 


analytic 
 5 


analytical 
 5 


analogy 
 6 


analogous 
 33 


analogue 
 12 


analphabet 
 1 


anamnesis 
 1 


anamorphic 
 1 


anamorphosis 
 1 


anaphase 
 3 


anaphylaxis 
 1 


anaphora 
 1 


anaphrodisiac 
 1 


anarchy 
 2 


anarchism 
 1 


anarchist 
 1 


anastigmatic 
 2 


anastomosis 
 1 


anatomy 
 19 


anatomical 
 19 


anatomist 
 1 


ancestor 
 55 


ancestral 
 24 


ancestry 
 18 


anchor 
 17 


anchorage 
 2 


anchorite 
 2 


anchorman 
 1 


anchovy 
 2 


ancient 
 661 


anciently 
 1 


ancillary 
 1 


ancylostomiasis 
 1 


and 
 17067 


andalusite 
 1 


andante 
 1 


andesite 
 2 


andradite 
 5 


androgen 
 1 


androgynous 
 2 


anecdotal 
 1 


anecdote 
 3 


anemometer 
 1 


anemone 
 11 


anencephaly 
 1 


aneroid 
 1 


aneuploid 
 1 


aneurysm 
 2 



automated 
 6 


automatic 
 39 


automatically 
 41 


automation 
 5 


automaton 
 1 


autometry 
 1 


automobile 
 7 


automorphic 
 1 


automorphism 
 3 


automotive 
 1 


autonomy 
 6 


autonomic 
 11 


autonomous 
 16 


autonomously 
 1 


autophagy 
 1 


autoplasty 
 1 


autopolyploid 
 1 


autopsy 
 5 


autopsic 
 1 


autoptic 
 2 


autoradiograph 
 4 


autoregulation 
 1 


autoroute 
 1 


autoschediastic 
 2 


autoscopy 
 2 


autosite 
 1 


autosoterism 
 1 


autosuggestion 
 5 


autotetraploid 
 1 


autotype 
 1 


autotomy 
 4 


autotoxin 
 1 


autotransfusion 
 2 


autotroph 
 1 


autoxidation 
 2 


autumn 
 29 


autunite 
 1 


auxiliary 
 17 


avadavat 
 1 


avail 
 2 


availability 
 5 


available 
 88 


avalanche 
 7 


avarice 
 1 


avaricious 
 2 


avascular 
 1 


avatar 
 2 


avenge 
 1 


avens 
 2 


avenue 
 2 


average 
 24 


averse 
 4 


aversion 
 21 


avert 
 3 


avian 
 2 


aviary 
 1 


aviation 
 5 


aviator 

bombing 
 5 


bon 
 1 


bond 
 33 


bonded 
 39 


bonding 
 4 


bondsman 
 2 


bone 
 281 


boned 
 3 


bonefish 
 1 


boneless 
 2 


bonfire 
 1 


bony 
 83 


bonito 
 1 


bonnet 
 3 


bonnethead 
 1 


bontebok 
 1 


bonus 
 4 


boo 
 1 


boobook 
 3 


book 
 264 


bookbinder 
 1 


bookcase 
 1 


booking 
 1 


bookish 
 1 


bookkeeping 
 1 


booklet 
 6 


bookmaker 
 1 


bookmark 
 1 


bookshelf 
 1 


bookshop 
 2 


boom 
 6 


boomerang 
 1 


booming 
 3 


boondocks 
 1 


boorish 
 2 


boost 
 2 


boot 
 35 


booted 
 1 


booth 
 4 


boots 
 15 


boracic 
 1 


borage 
 11 


borate 
 12 


borax 
 2 


border 
 87 


borderland 
 1 


borderline 
 2 


bore 
 15 


boreal 
 1 


bored 
 9 


boredom 
 4 


borehole 
 2 


borer 
 1 


boric 
 3 


boring 
 18 


born 
 69 


borne 
 33 


boron 
 9 


borosilicate 
 1 


borough 
 25 


borrow 
 3 


borrower 
 2 


borrowing 
 3 


bosom 
 2 


boson 
 2 


boss 
 6 


bossy 
 2 


botany 
 17

 3 


catarrh 
 3 


catarrhine 
 1 


catastrophe 
 2 


catastrophic 
 3 


catastrophism 
 2 


catatonia 
 1 


catch 
 50 


catcher 
 6 


catchy 
 1 


catching 
 35 


catchment 
 2 


catechin 
 1 


catechism 
 3 


catechol 
 2 


catechu 
 5 


category 
 59 


categorical 
 5 


categorically 
 1 


categorization 
 1 


categorize 
 1 


catenary 
 2 


catenative 
 1 


caterer 
 2 


catering 
 6 


caterpillar 
 23 


catfish 
 8 


catgut 
 1 


catharsis 
 1 


cathartic 
 1 


cathedral 
 23 


catheter 
 4 


cathexis 
 1 


cathode 
 19 


cation 
 18 


catkin 
 1 


catlike 
 3 


catlinite 
 1 


catmint 
 2 


catnip 
 1 


catty 
 1 


cattle 
 100 


catwalk 
 1 


caudal 
 12 


caudate 
 2 


caught 
 31 


cauldron 
 1 


cauliflower 
 4 


causal 
 3 


causality 
 1 


causation 
 9 


causative 
 9 


cause 
 713 


causeway 
 4 


caustic 
 7 


cautery 
 1 


caution 
 12 


cautionary 
 2 


cautious 
 17 


cautiously 
 3 


cava 
 6 


cavalry 
 2

...

sliding 
 25 


slight 
 62 


slightly 
 131 


slim 
 8 


slime 
 19 


slimy 
 9 


sling 
 4 


slingshot 
 1 


slink 
 2 


slip 
 8 


slipper 
 8 


slippery 
 8 


slipperiness 
 1 


slipping 
 6 


slipstream 
 1 


slipway 
 1 


slit 
 15 


sliver 
 2 


sloe 
 3 


slogan 
 4 


slope 
 54 


sloped 
 1 


sloping 
 66 


sloppy 
 2 


slot 
 8 


sloth 
 6 


slotted 
 4 


sloven 
 1 


slovenly 
 4 


slow 
 87 


slowcoach 
 1 


slowly 
 54 


slowness 
 1 


slowpoke 
 1 


slub 
 1 


sludge 
 1 


slug 
 3 


sluggish 
 7 


sluice 
 3 


slum 
 5 


slung 
 6 


slur 
 1 


slush 
 2 


slushy 
 2 


slut 
 1 


smack 
 2 


small 
 2489 


smallholder 
 2 


smallholding 
 1 


smallish 
 2 


smallness 
 3 


smallpox 
 12 


smalt 
 2 


smaltite 
 1 


smart 
 11 


smartly 
 1 


smartweed 
 1 


smash 
 1 


smear 
 8 


smectic 
 1 


smectite 
 3 


smell 
 114 


smelly 
 1 


smelt 
 4 


smelter 
 1 


smew 
 1 


smidgen 
 1 


smilax 
 1 


smile 


 5 


strangle 
 2 


strangulation 
 1 


strap 
 29 


strapless 
 2 


strapped 
 3 


strata 
 40 


stratagem 
 1 


strategy 
 20 


strategic 
 7 


strategically 
 1 


strategist 
 2 


strategos 
 1 


stratification 
 2 


stratigraphy 
 2 


stratosphere 
 6 


stratospheric 
 1 


stratum 
 24 


stratus 
 1 


straw 
 30 


strawberry 
 4 


streak 
 7 


streaky 
 6 


stream 
 75 


streamer 
 2 


streaming 
 3 


streamline 
 1 


street 
 52 


strength 
 173 


strengthen 
 21 


strenuous 
 10 


strenuously 
 1 


streptococcus 
 3 


streptomycete 
 1 


streptomycin 
 1 


stress 
 50 


stressed 
 20 


stressful 
 2 


stretch 
 35 


stretcher 
 2 


stretchy 
 3 


streusel 
 1 


strew 
 4 


strewn 
 3 


striated 
 13 


striatum 
 3 


strychnine 
 3 


strict 
 54 


strictly 
 34 


strictness 
 4 


stricture 
 1 


stride 
 7 


strife 
 3 


strike 
 58 


striker 
 1 


striking 
 69 


strikingly 
 10 


string 
 43 


stringboard 
 2 


stringed 

 3 


thankful 
 2 


thanks 
 5 


thanksgiving 
 2 


thar 
 1 


that 
 6251 


thataway 
 1 


thatch 
 2 


thatched 
 4 


thaumatrope 
 1 


thaw 
 3 


thawing 
 3 


the 
 32762 


theatre 
 64 


theatrical 
 37 


theatrically 
 2 


theca 
 2 


thecodont 
 1 


thee 
 2 


theft 
 12 


they 
 259 


their 
 932 


theirs 
 1 


theism 
 2 


them 
 329 


thematic 
 1 


thematically 
 1 


theme 
 31 


themselves 
 44 


then 
 101 


thenceforth 
 1 


theobromine 
 1 


theocracy 
 1 


theodolite 
 4 


theology 
 42 


theologian 
 9 


theological 
 18 


theorbo 
 1 


theorem 
 9 


theoretic 
 1 


theoretical 
 27 


theoretically 
 3 


theoretician 
 1 


theory 
 270 


theorize 
 5 


theorizing 
 1 


theosophy 
 2 


therapeutic 
 23 


therapy 
 29 


therapist 
 1 


there 
 147 


thereafter 
 1 


thereby 
 18 


therefore 
 50 


therian 
 1 


thermal 
 11 


thermionic 
 10 


thermoelectric 
 1 


thermogenesis 
 1 


thermograph 
 1 


thermomete

 6 


underhand 
 16 


underhandedness 
 1 


underlie 
 1 


underlying 
 43 


underling 
 1 


undermine 
 4 


undermining 
 3 


underneath 
 17 


undernourished 
 2 


undernourishment 
 1 


underpay 
 1 


underpants 
 10 


underpart 
 2 


underpinning 
 1 


undersea 
 1 


undersell 
 1 


undershirt 
 2 


undershoot 
 1 


undershot 
 1 


underside 
 42 


undersized 
 2 


underskirt 
 1 


understand 
 97 


understandable 
 1 


understandably 
 1 


understanding 
 76 


understated 
 3 


understatement 
 1 


understood 
 49 


undersurface 
 10 


undertake 
 16 


undertaker 
 2 


undertaking 
 40 


undertone 
 1 


undervalue 
 4 


underwater 
 7 


underwear 
 8 


underworld 
 24 


underwrite 
 1 


underwriter 
 1 


undeserved 
 1 


undeserving 
 3 


undesirable 
 58 


undetermined 
 3 


undeveloped 
 6 


undyed 
 2 


undifferentiated 
 5 


undigested 
 2 


undiluted 
 1 


undisciplined 
 2 


undiscovered 
 1 


undisguised 
 3 


undisputed 

 12 


waitress 
 5 


wayward 
 2 


wayzgoose 
 1 


wake 
 9 


wakeful 
 1 


waken 
 1 


waking 
 5 


wale 
 1 


walk 
 85 


walker 
 5 


walkway 
 9 


wall 
 269 


wallaby 
 3 


walled 
 8 


wallet 
 3 


wallflower 
 1 


wallow 
 2 


wallpaper 
 9 


walnut 
 4 


walrus 
 4 


waltz 
 5 


wampum 
 1 


wand 
 1 


wander 
 6 


wanderer 
 2 


wandering 
 9 


want 
 6 


wanting 
 12 


wanton 
 4 


wantonly 
 2 


wapiti 
 1 


war 
 77 


warble 
 2 


warbler 
 18 


ward 
 7 


warden 
 5 


warder 
 2 


wardrobe 
 5 


ware 
 11 


warehouse 
 7 


warfare 
 21 


warhead 
 1 


warhorse 
 1 


wary 
 3 


warlike 
 10 


warm 
 153 


warmer 
 8 


warmly 
 1 


warmth 
 24 


warn 
 8 


warning 
 40 


warp 
 13 


warped 
 2 


warrant 
 6 


warranty 
 3 


warrior 
 7 


warship 
 23 


wart 
 2 


wartime 
 6 


was 
 641 


wasabi 
 1 


wash 
 16 


washbasin 
 4 


washcloth 
 1 


washer 
 1 


washerman 
 1 


washerwoman 
 1 


washing 
 55 


w

In [13]:
for word in Core_graph:
    Core_defining_power[word] = 0
    for value in dicty.values():
        if word in value:
            Core_defining_power[word] += 1    

In [14]:
for word in Dictionary_graph:
    Dictionary_defining_power[word] = 0
    for value in dicty.values():
        if word in value:
            Dictionary_defining_power[word] += 1    

In [16]:
Core_defining_probabilities = Core_defining_power.copy()
Kernel_defining_probabilities = Kernel_defining_power.copy()
Dictionary_defining_probabilities = Dictionary_defining_power.copy()

for key in Core_defining_probabilities.keys():
    Core_defining_probabilities[key] /= 85183

for key in Kernel_defining_probabilities.keys():
    Kernel_defining_probabilities[key] /= 85183
    
for key in Dictionary_defining_probabilities.keys():
    Dictionary_defining_probabilities[key] /= 85183
    
save_obj(Dictionary_defining_power, 'Dictionary_defining_power')
save_obj(Kernel_defining_power, 'Kernel_defining_power')
save_obj(Core_defining_power, 'Core_defining_power')
    
save_obj(Dictionary_defining_probabilities, 'Dictionary_defining_probabilities')
save_obj(Kernel_defining_probabilities, 'Kernel_defining_probabilities')
save_obj(Core_defining_probabilities, 'Core_defining_probabilities')

In [17]:
def ShannonsCalculation(dictionary):
    information = 0
    for word, probability in dictionary.items():
        try:
            information += probability * math.log(probability, 2)
        except ValueError:
            pass
    return -(information)

print(ShannonsCalculation(Core_defining_probabilities))
print(ShannonsCalculation(Kernel_defining_probabilities))
print(ShannonsCalculation(Dictionary_defining_probabilities))

31.83412502338212
66.46740151746502
66.63544695964971


# Visualization Construction

In [49]:
from random import shuffle

word_list = [word for word in dicty.keys()]
shuffle(word_list)
mini_dict_list = word_list[:1000]

In [50]:
# mini_dict = {}
# for word in mini_dict_list:
#     mini_dict[word] = dicty[word]

# mini_dict_graph = nx.DiGraph()
# mini_dict_graph.add_nodes_from(mini_dict)
    
# createEdgesFromDefinitions(mini_dict_graph, mini_dict)

# list_of_nodes = mini_dict_graph.nodes
# list_of_edges = mini_dict_graph.edges

# words_to_remove = []

# # this process will reduce the dictionary graph to the kernel
# for word in mini_dict_graph:
#     # contained is a flag; we set it to False at first
#     contained = False
#     # iterate through each edge
#     for key, value in dicty.items():
#         # if the current word in the iteration is in the the definition of another word,
#         # flag contained = True and break out of the innermost loop
#         if word in value:
#             contained = True
#             break
#     # otherwise, add the word to the list of words to remove
#     if contained == False:
#         words_to_remove.append(word)

# mini_kernel_graph = mini_dict_graph.copy()

# # remove words that don't define other words from graph
# for word in words_to_remove:
#     mini_kernel_graph.remove_node(word)
    
# mini_core_graph = nx.DiGraph()
    
# for (word_1, word_2) in mini_kernel_graph.edges:
#         if word_1 in dicty[word_2] and word_2 in dicty[word_1]:
#             mini_core_graph.add_node(word_1)
#             mini_core_graph.add_node(word_2)
#             mini_core_graph.add_edge(word_1, word_2)
#             mini_core_graph.add_edge(word_2, word_1)



In [52]:
print(len(mini_dict_graph), len(mini_kernel_graph), len(mini_core_graph))
print(len(mini_dict_graph.edges), len(mini_kernel_graph.edges), len(mini_core_graph.edges))

1000 348 4
143 64 4


In [57]:
dictionary_words = mini_dict_graph.nodes - mini_kernel_graph.nodes
kernel_words = mini_kernel_graph.nodes - mini_core_graph.nodes
core_words = mini_core_graph.nodes
print(len(dictionary_words), len(kernel_words), len(core_words))

master_words = []
for word in dictionary_words:
    master_words.append((word, 0))
for word in kernel_words:
    master_words.append((word, 1))
for word in core_words:
    master_words.append((word, 2))


652 344 4


In [58]:
# Create a json object to represent the graph
nodes = [{'name': i[0], 'type': i[1]}
            for i in master_words]
links = [{'source': u[0], 'target': u[1]}
         for u in mini_dict_graph.edges]
with open('graph.json', 'w') as f:
    json.dump({'nodes': nodes, 'links': links},
              f, indent=4,)